In [6]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [8]:
url = 'https://raw.githubusercontent.com/leekyuyoung20230313/pythone/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/data/bike.csv'
bike = pd.read_csv(url)
bike.head()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1.0,0.0,1.0,0.0,6.0,0.0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1.0,0.0,1.0,0.0,0.0,0.0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1.0,0.0,1.0,0.0,1.0,1.0,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1.0,0.0,1.0,0.0,2.0,1.0,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1.0,0.0,1.0,0.0,3.0,1.0,1,0.226957,0.229270,0.436957,0.186900,1600


In [9]:
X = bike.iloc[:,1:-1]
y = bike.iloc[:,-1]

In [12]:
X.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
dtype: int64

In [17]:
# 모델
dt = DecisionTreeRegressor(random_state=100)
# 해당 모델을 이용한 교차검증
from sklearn.model_selection import cross_val_score
import sklearn

In [18]:
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [20]:
sv = cross_val_score(dt, X, y,scoring = 'neg_mean_squared_error')
sv # 5개가 나옴.

array([-1243738.08163265,  -641841.28767123, -2258767.04794521,
       -1601199.3630137 , -2062262.69178082])

In [ ]:
# cv는 분할기.

In [ ]:
# cross validation , cross_val_score  다 같은데 얘는 fit 안 해도 됨.

In [22]:
# RMSE
rmse = np.sqrt(-sv)
rmse

array([1115.23005772,  801.14997826, 1502.91950814, 1265.38506511,
       1436.05803914])

In [23]:
np.mean(rmse)

1224.148529674594

In [24]:
# 데이터 분할
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=100)

In [25]:
# 학습
dt.fit(x_train,y_train)

DecisionTreeRegressor(random_state=100)

In [26]:
# 예측
y_pred = dt.predict(x_train)
# rmse
from sklearn.metrics import mean_squared_error
dt_mse = mean_squared_error(y_train,y_pred)
dt_rmse = np.sqrt(dt_mse)
print(f'rmse: {dt_rmse}')

rmse: 0.0


In [27]:
# 학습하지 않은 테스트 데이터 예측
y__test_pred = dt.predict(x_test)
# rmse
from sklearn.metrics import mean_squared_error
dt_mse = mean_squared_error(y_test,y__test_pred) 
dt_rmse = np.sqrt(dt_mse)
print(f'rmse: {dt_rmse}')

rmse: 842.5550306014657


In [28]:
dt.score(x_train,y_train), dt.score(x_test,y_test) #오버핏임. 트리가 너무 깊어서 그럼.

(1.0, 0.7917927132755787)

In [29]:
# deci~는 깊이를 지정 안하면 계속 깊이가 깊어짐.
# 그러므로 뎁스 길이 조정하기.
# 근데 여기서 조정 안하고 하이퍼 파라메터로 할 거임.
dt.get_depth()

19

In [31]:
dt.feature_importances_, dt.feature_names_in_

(array([0.07949201, 0.27761082, 0.02195797, 0.00108578, 0.01084614,
        0.00207843, 0.04023998, 0.43094774, 0.04841651, 0.05618582,
        0.03113881]),
 array(['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday',
        'weathersit', 'temp', 'atemp', 'hum', 'windspeed'], dtype=object))

In [32]:
import_dt = dict(zip(dt.feature_names_in_,dt.feature_importances_))
import_dt = sorted(import_dt.items(), key= lambda x : x[1], reverse=True)
import_dt

# 얘네를 성능을 높이기 위해 하이퍼 튜닝들어감

[('temp', 0.4309477412411022),
 ('yr', 0.27761081761315914),
 ('season', 0.0794920107727855),
 ('hum', 0.05618582334147451),
 ('atemp', 0.04841650736778333),
 ('weathersit', 0.040239978047628445),
 ('windspeed', 0.031138806704320735),
 ('mnth', 0.021957970030936698),
 ('weekday', 0.010846137350116554),
 ('workingday', 0.0020784259097937326),
 ('holiday', 0.0010857816208990588)]

In [35]:
# 파이퍼 파라메터 튜닝
from sklearn.model_selection import GridSearchCV
# 모델 로드
dt = DecisionTreeRegressor(random_state=100)
# 파라메터
params = {'max_depth' : [None,2,3,4,5,6,7,8,9,10,20]}
# 튜닝
grid_reg = GridSearchCV(dt,param_grid=params,scoring='neg_mean_squared_error',cv=5,return_train_score=True, n_jobs=-1 )
grid_reg.fit(x_train,y_train)
# 최상의 매개변수 출력
print(f'최상의 매개변수 : {grid_reg.best_params_}')

최상의 매개변수 : {'max_depth': 5}


In [37]:
best_model = grid_reg.best_estimator_
y_pred = best_model.predict(x_test)
# rmse
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
print(f'test rmse : {rmse}')

test rmse : 747.2054971052871


In [38]:
best_model.feature_importances_

array([8.75965786e-02, 3.12334392e-01, 1.73185454e-02, 1.20460074e-03,
       2.53858443e-04, 0.00000000e+00, 5.01818716e-02, 4.59825448e-01,
       2.97637479e-02, 2.47637663e-02, 1.67571912e-02])

In [42]:
import_dt = dict(zip(best_model.feature_names_in_, best_model.feature_importances_))
import_dt = sorted(import_dt.items(),key = lambda x : x[1],reverse=True)
import_dt

[('temp', 0.45982544809130715),
 ('yr', 0.3123343917420042),
 ('season', 0.08759657863333012),
 ('weathersit', 0.050181871585009294),
 ('atemp', 0.029763747927659116),
 ('hum', 0.02476376627935139),
 ('mnth', 0.017318545362041037),
 ('windspeed', 0.016757191200028755),
 ('holiday', 0.0012046007361451148),
 ('weekday', 0.0002538584431237691),
 ('workingday', 0.0)]

In [46]:
from sklearn.inspection import permutation_importance
# permutation_importance 순열 중요도.. feature 특성 - 각 특성을 무작위로 섞어서 모델의 성능 변화를 측정

result = permutation_importance(best_model,x_train,y_train,random_state=100)
import_dt = dict(zip(x_train.columns, result.importances_mean))
import_dt = sorted(import_dt.items(),key = lambda x : x[1],reverse=True)
import_dt

[('yr', 0.5612204617244092),
 ('temp', 0.5091022401828734),
 ('season', 0.11425871259463789),
 ('weathersit', 0.08484595603974827),
 ('atemp', 0.0781860416330136),
 ('hum', 0.07583963802114117),
 ('mnth', 0.043045582128536174),
 ('windspeed', 0.037821758596693324),
 ('holiday', 0.0019205556236377096),
 ('weekday', 0.0005415261474897903),
 ('workingday', 0.0)]

In [ ]:
# 인공 신경망 전에 머신러닝을 내일까지 보고, 인공신경망.
# 이 다음엔 대